In [ ]:
"""ipynb for hiperparameter optimalization. See log in /log/ dir."""

from ray import tune 
from ray import tune, air
from ray.air import session
from ray.tune.search.optuna import OptunaSearch
import optuna
import initialize 
import model
from PIL import Image
import numpy as np
from torchvision import transforms 
import torchvision
import torch
import ray
import torch.optim as optim

device = torch.device("cpu")


In [2]:
def train_velmodel(model_vel,optimizer_vel,trainloader_vel, criterion, testloader_vel):
    out=0
    for epochs in range(5): 
        running_loss = 0.0
        for i, data in enumerate(trainloader_vel, 0):
            # get the inputs; data is a list of [inputs, labels]
            x_train, y_train = data[0], data[1]
            # zero the parameter gradients
            optimizer_vel.zero_grad()

            # forward + backward + optimize
            outputs = model_vel(x_train)
            loss = criterion(outputs, y_train)
            loss.backward()
            optimizer_vel.step()

            # print statistics
            running_loss += loss.item()
            if i % 100 == 99:    
                print(f'[{epochs + 1}, {i + 1:5d}] loss: {running_loss / 10:.3f}')
                running_loss = 0.0
                correct = 0
                total = 0
                with torch.no_grad():
                    for data2 in testloader_vel:
                        x_test, y_test = data2[0].to(device), data2[1].to(device)   
                        outputs = model_vel(x_test)
                        _, predicted = torch.max(outputs.data,1)
                        _,right=torch.max(y_test,1)
                        total += y_test.size(0)
                        correct += (predicted == right).sum().item()
                        #print(f'Accuracy of the network on the 2000 test images: {100*correct // total} %')
                        out=correct/total
    
    return out 


In [3]:

def train_angmodel(model_ang,trainloader_ang, optimizer_ang,criterion):
    for epochs in range(10): 
        running_loss = 0.0
        for i, data in enumerate(trainloader_ang, 0):
            # get the inputs; data is a list of [inputs, labels]
            x_train, y_train2 = data[0].to(device), data[1].to(device)
            # zero the parameter gradients
            optimizer_ang.zero_grad()

            # forward + backward + optimize
            outputs = model_ang(x_train)
        
            loss = criterion(outputs, y_train2)
            loss.backward()
            optimizer_ang.step()

            # print statistics
            running_loss += loss.item()
            if i % 10 == 9:    # print every 2000 mini-batches
                print(f'[{epochs + 1}, {i + 1:5d}] loss: {running_loss / 10:.3f}')
                running_loss = 0.0
                correct = 0
                total = 0

    path="/home/axelk/Documents/DL/angle.pth"
    torch.save(model_ang,path )

In [4]:

def objective(config):
    trainloader_vel, k = initialize.load_data(100,32, True)
    testloader_vel, k2= initialize.load_data(100,32, False)
    model_vel = model.veloCNN().to("cpu")

    criterion=torch.nn.CrossEntropyLoss()

    optimizer_vel = torch.optim.SGD( model_vel.parameters(), lr=config["lr"], momentum=config["momentum"])

    while True:
        acc = train_velmodel(model_vel, optimizer_vel, trainloader_vel,criterion,testloader_vel)  # Train the model
        h= np.array([acc, config["lr"], config["momentum"]])
        session.report({"mean_accuracy": acc})  # Report to Tune


In [5]:

search_space = {"lr": tune.loguniform(1e-4, 2e-1), "momentum": tune.uniform(0.1, 0.9)}
algo = OptunaSearch()


In [6]:

tuner = tune.Tuner(
    
    objective,
    tune_config=tune.TuneConfig(
        metric="mean_accuracy",
        mode="max",
        search_alg=algo,
    ),
    run_config=air.RunConfig(
        stop={"training_iteration": 100},
    ),
    param_space=search_space,

)
results = tuner.fit()
print("Best config is:", results.get_best_result().config)


2022-12-11 16:40:57,110	INFO worker.py:1528 -- Started a local Ray instance.
2022-12-11 16:40:57,707	WARNING function_trainable.py:586 -- Function checkpointing is disabled. This may result in unexpected behavior when using checkpointing features or certain schedulers. To enable, set the train function arguments to be `func(config, checkpoint_dir=None)`.
/home/axelk/.local/lib/python3.10/site-packages/ray/tune/search/optuna/optuna_search.py:673: FutureWarning: LogUniformDistribution has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :class:`~optuna.distributions.FloatDistribution` instead.
  return ot.distributions.LogUniformDistribution(
/home/axelk/.local/lib/python3.10/site-packages/ray/tune/search/optuna/optuna_search.py:682: FutureWarning: UniformDistribution has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :class:`~o

/home/axelk/.local/lib/python3.10/site-packages/optuna/distributions.py:766: FutureWarning: LogUniformDistribution(high=0.2, low=0.0001) is deprecated and internally converted to FloatDistribution(high=0.2, log=True, low=0.0001, step=None). See https://github.com/optuna/optuna/issues/2941.
  warnings.warn(message, FutureWarning)
/home/axelk/.local/lib/python3.10/site-packages/optuna/distributions.py:766: FutureWarning: UniformDistribution(high=0.9, low=0.1) is deprecated and internally converted to FloatDistribution(high=0.9, log=False, low=0.1, step=None). See https://github.com/optuna/optuna/issues/2941.
  warnings.warn(message, FutureWarning)


(objective pid=559150) (15996, 3, 75, 100)
(objective pid=559150) (15996, 4)
(objective pid=559150) (15996, 3)
(objective pid=559150) (2001, 3, 75, 100)
(objective pid=559150) (2001, 4)
(objective pid=559150) (2001, 3)
(objective pid=559150) [1,   100] loss: 12.637
(objective pid=559150) [1,   200] loss: 11.337
(objective pid=559150) [1,   300] loss: 10.541
(objective pid=559150) [1,   400] loss: 10.223
(objective pid=559150) [1,   500] loss: 9.748
(objective pid=559150) [2,   100] loss: 9.408
(objective pid=559150) [2,   200] loss: 9.018
(objective pid=559150) [2,   300] loss: 8.878
(objective pid=559150) [2,   400] loss: 8.562
(objective pid=559150) [2,   500] loss: 8.602
(objective pid=559150) [3,   100] loss: 8.202
(objective pid=559150) [3,   200] loss: 8.271
(objective pid=559150) [3,   300] loss: 8.082
(objective pid=559150) [3,   400] loss: 7.726
(objective pid=559150) [3,   500] loss: 7.957
(objective pid=559150) [4,   100] loss: 7.586
(objective pid=559150) [4,   200] loss: 7

Trial name,date,done,episodes_total,experiment_id,hostname,iterations_since_restore,mean_accuracy,node_ip,pid,time_since_restore,time_this_iter_s,time_total_s,timestamp,timesteps_since_restore,timesteps_total,training_iteration,trial_id,warmup_time
objective_34c5ae48,2022-12-11_17-38-40,True,,d519957b3b6a4e7db7620f6dbe1b90fa,axelk,100,0.763118,192.168.0.107,559150,3461.3,34.7278,3461.3,1670776720,0,,100,34c5ae48,0.00174642


(objective pid=559150) [1,   100] loss: 6.689
(objective pid=559150) [1,   200] loss: 6.785
(objective pid=559150) [1,   300] loss: 6.881
(objective pid=559150) [1,   400] loss: 6.820
(objective pid=559150) [1,   500] loss: 6.840
(objective pid=559150) [2,   100] loss: 6.325
(objective pid=559150) [2,   200] loss: 6.478
(objective pid=559150) [2,   300] loss: 6.587
(objective pid=559150) [2,   400] loss: 6.528
(objective pid=559150) [2,   500] loss: 6.534
(objective pid=559150) [3,   100] loss: 6.213
(objective pid=559150) [3,   200] loss: 6.214
(objective pid=559150) [3,   300] loss: 6.204
(objective pid=559150) [3,   400] loss: 6.312
(objective pid=559150) [3,   500] loss: 5.969
(objective pid=559150) [4,   100] loss: 5.800
(objective pid=559150) [4,   200] loss: 5.837
(objective pid=559150) [4,   300] loss: 5.804
(objective pid=559150) [4,   400] loss: 5.959
(objective pid=559150) [4,   500] loss: 5.931
(objective pid=559150) [5,   100] loss: 5.385
(objective pid=559150) [5,   200] 

2022-12-11 17:38:40,746	INFO tune.py:777 -- Total run time: 3463.04 seconds (3462.92 seconds for the tuning loop).


Best config is: {'lr': 0.0005369478633254896, 'momentum': 0.6780438988661533}
